In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe', 'alcool', 'acido_malico', 'cinzas', 'alcalinidade_de_cinzas', 'magnesio', 
    'fenois_totais', 'flavanoides', 'fenois_nao_flavanoides', 'proantocianinas', 
    'intensidade_de_cor', 'matiz', 'od280_od315_de_vinhos_diluidos', 'prolina'
]

vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Separando características (X) e rótulos (y)
X = vinhos.drop('classe', axis=1)
y = vinhos['classe']

def knn_kfold_experiment(random_state, k_values):
    # Configuração do KFold com 10 folds
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    
    results = {k: [] for k in k_values}
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            results[k].append(accuracy)
    
    for k in k_values:
        accuracies = results[k]
        mean_accuracy = np.mean(accuracies)
        std_accuracy = np.std(accuracies)
        print(f"Random state: {random_state}, k={k}, Acurácia média: {mean_accuracy:.4f}, Desvio padrão: {std_accuracy:.4f}")
    
    return results

k_values = [3, 5]

resultados_42 = knn_kfold_experiment(random_state=42, k_values=k_values)
resultados_17 = knn_kfold_experiment(random_state=17, k_values=k_values)
resultados_24 = knn_kfold_experiment(random_state=24, k_values=k_values)

def analisar_melhor_k(resultados, random_state):
    medias = {k: np.mean(resultados[k]) for k in resultados}
    melhor_k = max(medias, key=medias.get)  # k com maior acurácia média
    print(f"\nPara random_state={random_state}, o melhor valor de k foi: {melhor_k}\n")

analisar_melhor_k(resultados_42, 42)
analisar_melhor_k(resultados_17, 17)
analisar_melhor_k(resultados_24, 24)
